<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/hate_speech/data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation

Cloning into 'Objective-criterias-to-quantify-the-accuracy-of-explanation'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 294 (delta 90), reused 137 (delta 62), pack-reused 101
Receiving objects: 100% (294/294), 23.30 MiB | 6.96 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Updating files: 100% (21/21), done.


In [2]:
import pandas as pd
import sys,json,pickle,re
sys.path.append('Objective-criterias-to-quantify-the-accuracy-of-explanation/hate_speech')
from utils import *

In [3]:
data_path = 'Objective-criterias-to-quantify-the-accuracy-of-explanation/hate_speech/Dataset'
train_file = data_path+'/data.csv'

In [4]:
train_df = pd.read_csv(train_file)
print(train_df.head())

   Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   

                                                Text  Label  
0                मैं अपने लिए तो अपने आप बनाती हूँ |      2  
1  और वो लोग अपने लिए अपने आप बनाते है उनकी फॅमिली |      2  
2  में अलग हो गया था अलग हो गया था सब मतलब 1 साल ...      1  
3  नहीं वो है बड़ी वाली तो जो उनकी बेटी है और जो म...      0  
4  मै मेरको ना (english : actually time ) लगता था...      1  


In [5]:
# 0 - hate speech 1 - offensive language 2 - neither as found in documentation
labels = list(train_df["Label"].unique())
print(labels)

[2, 1, 0]


In [6]:
label2ind = {"Introduction":0,"Depression":1,"Grey area":2}
ind2label = {i:l for l,i in label2ind.items()}

In [7]:
with open(data_path+"/label2ind.json","w") as f:
    json.dump(label2ind,f,indent=1)
with open(data_path+"/ind2label.json","w") as f:
    json.dump(ind2label,f,indent=1)

In [8]:
new_train_df = train_df[["Text","Label"]]
label_to_category = {0: 'Introduction', 1: 'Depression', 2: 'Grey area'}
new_labels = new_train_df['Label'].map(label_to_category)
text = new_train_df['Text'][:]
new_train_df = pd.DataFrame({'Text':text, "Label":new_labels})
new_train_df

,Text,Label
0,मैं अपने लिए तो अपने आप बनाती हूँ |,Grey area
1,और वो लोग अपने लिए अपने आप बनाते है उनकी फॅमिली |,Grey area
2,में अलग हो गया था अलग हो गया था सब मतलब 1 साल ...,Depression
3,नहीं वो है बड़ी वाली तो जो उनकी बेटी है और जो म...,Introduction
4,मै मेरको ना (english : actually time ) लगता था...,Depression
...,...,...
6494,"हे ना , वो इतना पढ़ी - लिखी थी वो मेरे से बेटर...",Depression
6495,"मेरे ये था में मतलब मुझे , अभी भी ऐसा हो रहा ह...",Depression
6496,जैसे मुझे कोई चीज़ अगर ऐसी वैसी लगती है कोई भी ...,Depression
6497,"हाँ, उसके बाद में ही मुझे सेटिस्फैक्शन फील ( e...",Depression


In [9]:
train_file_path = data_path+"/hate_speech_train.csv"
new_train_df.to_csv(train_file_path,sep=",")

In [10]:
train_csv_file = data_path+"/hate_speech_train.csv"

In [11]:
def load_data(csv_path, n=-1): # if n=-1 it will load all data
    df = pd.read_csv(csv_path, delimiter=",")
    df = df.sample(frac=1)  # shuffling rows
    texts = []
    labels = []
    c = 0
    if n == -1:
        n = len(df)
    for idx in df.index:
        text = df["Text"][idx]
        label = df["Label"][idx]
        texts.append(text)
        labels.append(label)
        c += 1
        if c >= n:
            break
    return texts, labels

In [12]:
# load all data
all_texts,all_labels = load_data(train_csv_file,n=-1)

In [13]:
# split data in Train and Validation
train_data,val_data = split_for_train(all_texts,all_labels)
x_train,y_train = train_data[0],train_data[1]
x_val,y_val = val_data[0],val_data[1]
# Further split Train data in Test as Original Kaggle Test data does not have labels
val_data,test_data = split_for_train(x_val,y_val,val_split=0.2)
x_val,y_val = val_data[0],val_data[1]
x_test,y_test = test_data[0],test_data[1]
print("Total %d x_train, %d y_train"%(len(x_train),len(y_train)))
print("Total %d x_val, %d y_val"%(len(x_val),len(y_val)))
print("Total %d x_test, %d y_test"%(len(x_test),len(y_test)))

Total 4551 x_train, 4551 y_train
Total 1560 x_val, 1560 y_val
Total 388 x_test, 388 y_test


In [14]:
with open( data_path+'/train_pkl',"wb") as f:
    pickle.dump((x_train,y_train),f)
with open( data_path+'/test_pkl',"wb") as f:
    pickle.dump((x_test,y_test),f)
with open( data_path+'/validation_pkl',"wb") as f:
    pickle.dump((x_val,y_val),f)